In [5]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import func4behav as fb
from scipy.stats import lognorm, exponnorm, invgauss
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from pyddm import Model, Sample, Fittable, Fitted
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, ICPointSourceCenter, LossRobustLikelihood
from pyddm.functions import fit_adjust_model, display_model
import pyddm.plot

import imp
imp.reload(fb)

<module 'func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tes-attention\\processing\\behavior\\func4behav.py'>

In [6]:
def distribution_param(case_list, cutoff):

    _, expt_info = fb.create_allsubs_compare()
    param_record = pd.DataFrame(columns=['subject id', 'case', 'group', 'session', 'method', 'params', 'log likelihood', 'AIC'])
    
    session_name = ['before', 'after']
    methods_name = ['shift lognorm', 'exgaussian', 'ddm', 'shift invgauss']
    methods = [fb.fit_shift_lognorm, fb.fit_exgaussian, fb.fit_ddm, fb.fit_shift_invgauss]

    # Loops
    record_idx = 0
    for subject_id in range(1,19):
        print("Subject id:" + str(subject_id) + " is processing...")
        real_or_sham = expt_info.loc[expt_info['subject id'] == subject_id, 'Real stimulation'].values[0]

        for case in case_list:
            rt_before, rt_after = make_rt_data(subject_id, case, cutoff)

            for method_idx, method in enumerate(methods):
                for session_idx, rt in enumerate([rt_before, rt_after]):
                    param, log_L, AIC = method(rt, cutoff)

                    param_record.loc[record_idx, 'subject id'] = subject_id
                    param_record.loc[record_idx, 'case'] = case
                    param_record.loc[record_idx, 'group'] = "real" if real_or_sham == 1 else "sham"
                    param_record.loc[record_idx, 'session'] = session_name[session_idx]
                    param_record.loc[record_idx, 'method'] = methods_name[method_idx]
                    param_record.loc[record_idx, 'params'] = param
                    param_record.loc[record_idx, 'log likelihood'] = log_L
                    param_record.loc[record_idx, 'AIC'] = AIC

                    record_idx += 1
                    
    return param_record


def make_rt_data(subject_id, case, cutoff):
    
    behavior_before, behavior_after = fb.load_behavior(subject_id)
    behavior_before, behavior_after = fb.filter_behav(case, behavior_before, behavior_after)
    rt_before = behavior_before.loc[(behavior_before['response'] == 1) & (behavior_before['reaction time'] > 0.05) & (behavior_before['reaction time'] < cutoff)]
    rt_after = behavior_after.loc[(behavior_after['response'] == 1) & (behavior_after['reaction time'] > 0.05) & (behavior_after['reaction time'] < cutoff)]

    rt_before = rt_before['reaction time'].values
    rt_after = rt_after['reaction time'].values

    return rt_before, rt_after

In [7]:
all_real = [1, 3, 4, 5, 9, 12, 13, 17, 18]
all_sham = [2, 6, 7, 8, 10, 11, 14, 15, 16]
case_list = ['all', 'endo', 'exo', 'valid', 'endo valid', 'exo valid', 'invalid', 'endo invalid', 'exo invalid', 'stim left', 'endo stim left', 'endo stim right', 'stim right', 'exo stim left', 'exo stim right']
cutoff = 1
param_record = distribution_param(case_list, cutoff)

Subject id:1 is processing...


KeyboardInterrupt: 